- 0429 새로만든 훈련데이터 (balanced 엄격하게)

원본 : (중요)2020_0417_Training_with_difference_vector_wo_balance(Bagging-probX)_ver4_iter_estimators.ipynb

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.model_selection import train_test_split

import glob
import os
import multiprocessing

import pickle
import time
import random

---

In [2]:
def chunkIt(seq, num):
    if len(seq) % int(num) == 0:
        avg = int(len(seq) / int(num))
    else:
        avg = int(len(seq) / int(num)) + 1 # 이렇게 해야하네!
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg

    return out

---

# data load

### - train/test index

In [3]:
glob.glob('../2_svm2+data/data/without_validation/*')

['../2_svm2+data/data/without_validation/test_index.pkl',
 '../2_svm2+data/data/without_validation/train_sampling_dict.pkl',
 '../2_svm2+data/data/without_validation/train_pair.pkl',
 '../2_svm2+data/data/without_validation/train_index.pkl',
 '../2_svm2+data/data/without_validation/balanced',
 '../2_svm2+data/data/without_validation/without_balanced']

In [4]:
# load train/text index
with open('../2_svm2+data/data/without_validation/train_index.pkl', 'rb') as f:
    train_index = pickle.load(f)
    
with open('../2_svm2+data/data/without_validation/test_index.pkl', 'rb') as f:
    test_index = pickle.load(f)

---

### - load train_array

In [5]:
glob.glob('*')

['X_tain.pkl',
 'train_sampling_dict.pkl',
 '(중요)2020_0428_SVM_Bagging_Unbalanced(+new_diff).ipynb',
 '2020_0428_make_train_data_with_balanced(+wo_balanced+no_validation)+EDA.ipynb',
 'y_train.pkl',
 'ReadME.txt']

In [6]:
with open('X_tain.pkl', 'rb') as f:
    X_train = pickle.load(f)

In [7]:
with open('y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)

In [8]:
print(X_train.shape, y_train.shape)

(330519, 512) (330519,)


---

# data make

### - new-diff

 *** multiprocessing ***

In [22]:
# X , y 합쳐주기

train = np.append(X_train, y_train.reshape(len(X_train), -1), axis=1)

In [23]:
train_chunk = chunkIt(train, 36)

In [24]:
train_chunk[0].shape

(9182, 513)

In [25]:
def parallelize_dataframe(chunk_pair, func):
    #
    #ctx = multiprocessing.get_context("spawn")
    #pool  = ctx.Pool(num_cores)
    pool = multiprocessing.Pool( num_cores )
    stats_list = pool.map(func, chunk_pair) # 리스트 형태로 떨궈준다
    pool.close()
    pool.join()
    print('parallel finished !')
    return stats_list

In [26]:
def make_stats( chunk_pair ):
    
    data_chunk = chunk_pair
    
    # x1-x2 : X_diff   
    for i, x in enumerate( data_chunk ):
        if i == 0:
            X_diff = (x[:256] - x[256:512]).reshape(1, 256)
            y_chunk = x[512:]
        else:
            X_diff = np.append(X_diff, (x[:256] - x[256:512]).reshape(1, 256) , axis=0)
            y_chunk = np.append(y_chunk, x[512:] )
    
    # x2-x1 : X_diff_reversed
    for i, x in enumerate( data_chunk ):
        if i == 0:
            X_diff_reversed = (x[256:512] - x[:256]).reshape(1, 256)
            y_chunk_reversed = x[512:]
        else:
            X_diff_reversed = np.append(X_diff_reversed, (x[256:512] - x[:256]).reshape(1, 256) , axis=0)
            y_chunk_reversed = np.append(y_chunk_reversed, x[512:] )
            
    # append
    X_diff = np.append(X_diff, X_diff_reversed, axis=0)  
    y_chunk = np.append( y_chunk, y_chunk_reversed )
    
    return (X_diff, y_chunk)

- for Train

In [27]:
%%time

num_cores = len(train_chunk)  # 

# df_split은 위에 정의했다
train_parallel = parallelize_dataframe( train_chunk , make_stats )

parallel finished !
CPU times: user 6.35 s, sys: 11.1 s, total: 17.4 s
Wall time: 2min 53s


In [28]:
len(train_parallel[0][0])

18364

---

- 데이터 합치기

In [29]:
X_diff_train = np.array([])
y_diff_train = np.array([])
for i,(x,y) in enumerate( train_parallel ):
    if i == 0:
        X_diff_train = x
        y_diff_train = y
    else:
        X_diff_train = np.append( X_diff_train, x, axis=0 )
        y_diff_train = np.append( y_diff_train, y, axis=0 )

In [30]:
print(X_diff_train.shape, y_diff_train.shape )

(661038, 256) (661038,)


---
---

# # Training

In [10]:
import time
import numpy as np
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn import datasets
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

# # with rbf
- non-scaling

In [32]:
%%time
n_estimators = 30

estimator = SVC( max_iter = 100 ) # class_weight : 'balanced'로 해야할까?
bagging = BaggingClassifier(base_estimator = estimator, max_samples=1.0 / n_estimators, 
                        n_estimators=n_estimators, n_jobs=36)
bagging.fit(X_diff_train, y_diff_train)

CPU times: user 483 ms, sys: 4.27 s, total: 4.75 s
Wall time: 29.5 s


BaggingClassifier(base_estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                                     class_weight=None, coef0=0.0,
                                     decision_function_shape='ovr', degree=3,
                                     gamma='scale', kernel='rbf', max_iter=100,
                                     probability=False, random_state=None,
                                     shrinking=True, tol=0.001, verbose=False),
                  bootstrap=True, bootstrap_features=False, max_features=1.0,
                  max_samples=0.03333333333333333, n_estimators=30, n_jobs=36,
                  oob_score=False, random_state=None, verbose=0,
                  warm_start=False)

---

- 테스트 환경 일지 기록
> 기본적으로 n_estimators : 30, max_iter : 100, n_jobs : 36 으로 시작

> - kernel : linear -> 훈련시간 27초
> - kernel : rbf(default) -> 훈련시간 30초

# # with poly3

In [42]:
%%time
n_estimators = 30

estimator = SVC( kernel = 'poly' , max_iter = 100 ) # 
bagging_poly3 = BaggingClassifier(base_estimator = estimator, max_samples=1.0 / n_estimators, 
                        n_estimators=n_estimators, n_jobs=36)
bagging_poly3.fit(X_diff_train, y_diff_train)

CPU times: user 353 ms, sys: 580 ms, total: 933 ms
Wall time: 22.7 s


BaggingClassifier(base_estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                                     class_weight=None, coef0=0.0,
                                     decision_function_shape='ovr', degree=3,
                                     gamma='scale', kernel='poly', max_iter=100,
                                     probability=False, random_state=None,
                                     shrinking=True, tol=0.001, verbose=False),
                  bootstrap=True, bootstrap_features=False, max_features=1.0,
                  max_samples=0.03333333333333333, n_estimators=30, n_jobs=36,
                  oob_score=False, random_state=None, verbose=0,
                  warm_start=False)

---

## ## evaluate test set

- multiprocessing 활용하기 위해선, 훈련때 n_jobs=1로 해야해.. 
- bagging_1

In [14]:
test_index[:5]

[40648, 66052, 37783, 59761, 16639]

In [15]:
glob.glob('../1_svm/data/*.csv')

['../1_svm/data/new_feat_df.csv']

In [16]:
data = pd.read_csv('../1_svm/data/new_feat_df.csv')

In [17]:
group_no_df = data.groupby('group_no').size().reset_index()
group_no_df.columns = ['group_no', 'size_']

filtering_grp = group_no_df[group_no_df.size_ > 2].group_no.tolist()
data_filter = data[data.group_no.isin( filtering_grp )]

In [18]:
data_filter.shape

(73681, 266)

---
---

- 이런식으로 하면 될것같아
> (문제점!) 그치만 램을 쭉쭉 뽑아먹고, 커널이 죽는다 (test_index 1000개에 대한 pair 데이터 다 만들면)

- 테스트 pair 데이터 500개 정도 저장해서 사용하고 싶었는데, 30기가가 넘는것 같아
> - 일단은 100개 정도만 활용하자

In [19]:
%%time
# init
test_pair_array = {}
test_label = {}   
start_time = time.time()
# loop
for j,test_ in enumerate(test_index[:100]):
    
    # init
    test_pair_array[test_] = np.array([])
    test_label[test_] = np.array([])
    ###
    pair_array = data_filter.iloc[:, 7:-3].values
    label_array = data_filter.group_no.values
    
    anchor_array = pair_array[test_]
    anchor_label = data_filter.iloc[test_].group_no
    
    candidate_array = np.delete( pair_array, test_, axis=0)
    candidate_label = np.delete(label_array, test_)
    
    # append - diff 형태
    test_pair_array[test_] = (anchor_array - candidate_array)
    test_label[test_] = ( candidate_label == anchor_label ).astype(int)
    if (j+1) % 100 == 0:
        elapsed_time = time.time() - start_time
        print(elapsed_time)

17.7384512424469
CPU times: user 7.27 s, sys: 10.5 s, total: 17.7 s
Wall time: 17.7 s


In [33]:
len(test_pair_array)

100

----

- prediction 만들기

In [34]:
%%time
prediction_dict = {}

for key_ in list(test_label.keys())[:100]:
    prediction_dict[key_] = bagging.predict(test_pair_array[key_])

CPU times: user 13.6 s, sys: 10.5 s, total: 24.1 s
Wall time: 12min 12s


### max_iter & n_estimators 테스트
###<n_jobs:36>
- max_iter 100 & n_estimators 30 : 100개 테스트 객체 통해서 prediction 구할 때, 약 12분 소요 



In [35]:
len(prediction_dict)

100

In [36]:
from sklearn import metrics

- test_set에 대한 성능 살피기
> precision / recall

## # rbf(=default)

### 1. max_iter 100 & n_estimators 30

In [37]:
precision_100 = []
recall_100 = []

for ind, predicted in prediction_dict.items(): 
    print(ind)
    print('label_size : {}'.format(np.sum(test_label[ind])), 
          'prediction_size : {}'.format(np.sum(predicted)) )
    
    # calculate precision / recall
    precision_ = metrics.precision_score( test_label[ind] , predicted )
    recall_ = metrics.recall_score( test_label[ind] , predicted )
    
    # print
    print( precision_ )
    print( recall_ )
    print('\n')
    
    # append precision / recall
    precision_100.append( precision_ )
    recall_100.append( recall_ )
    

40648
label_size : 4 prediction_size : 1.0
1.0
0.25


66052
label_size : 2 prediction_size : 17.0
0.11764705882352941
1.0


37783
label_size : 3 prediction_size : 0.0
0.0
0.0


59761
label_size : 2 prediction_size : 2.0


/home/wmind/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1.0
1.0


16639
label_size : 4 prediction_size : 4.0
1.0
1.0


17958
label_size : 5 prediction_size : 5.0
1.0
1.0


10918
label_size : 5 prediction_size : 18.0
0.2777777777777778
1.0


26542
label_size : 4 prediction_size : 4.0
1.0
1.0


71651
label_size : 2 prediction_size : 2.0
1.0
1.0


45696
label_size : 3 prediction_size : 1.0
1.0
0.3333333333333333


4253
label_size : 8 prediction_size : 12.0
0.6666666666666666
1.0


63461
label_size : 2 prediction_size : 29.0
0.06896551724137931
1.0


572
label_size : 14 prediction_size : 17.0
0.8235294117647058
1.0


66460
label_size : 2 prediction_size : 0.0
0.0
0.0


70828
label_size : 2 prediction_size : 0.0
0.0
0.0


34741
label_size : 4 prediction_size : 3.0
1.0
0.75


14588
label_size : 7 prediction_size : 0.0
0.0
0.0


8647
label_size : 4 prediction_size : 4.0
0.25
0.25


51675
label_size : 3 prediction_size : 7.0
0.42857142857142855
1.0


44240
label_size : 3 prediction_size : 3.0
1.0
1.0


15772
label_size : 4 prediction_size : 8.0
0.5

In [38]:
print(len(precision_100), len(recall_100) )

100 100


In [39]:
np.mean(precision_100)

0.6250409527315884

In [40]:
np.mean(recall_100)

0.7175813492063492

---
---

# # bagging_poly3 결과

- prediction 만들기

In [ ]:
%%time
prediction_poly3_dict = {}

for key_ in list(test_label.keys())[:100]:
    prediction_poly3_dict[key_] = bagging_poly3.predict(test_pair_array[key_])

In [44]:
len(prediction_poly3_dict)

100

### 테스트 결과 (test_index[:100])

In [45]:
precision_100 = []
recall_100 = []

for ind, predicted in prediction_poly3_dict.items(): 
    print(ind)
    print('label_size : {}'.format(np.sum(test_label[ind])), 
          'prediction_size : {}'.format(np.sum(predicted)) )
    
    # calculate precision / recall
    precision_ = metrics.precision_score( test_label[ind] , predicted )
    recall_ = metrics.recall_score( test_label[ind] , predicted )
    
    # print
    print( precision_ )
    print( recall_ )
    print('\n')
    
    # append precision / recall
    precision_100.append( precision_ )
    recall_100.append( recall_ )
    

40648
label_size : 4 prediction_size : 73680.0
5.428881650380022e-05
1.0


66052
label_size : 2 prediction_size : 73680.0
2.714440825190011e-05
1.0


37783
label_size : 3 prediction_size : 73680.0
4.0716612377850165e-05
1.0


59761
label_size : 2 prediction_size : 73680.0
2.714440825190011e-05
1.0


16639
label_size : 4 prediction_size : 73680.0
5.428881650380022e-05
1.0


17958
label_size : 5 prediction_size : 73680.0
6.786102062975027e-05
1.0


10918
label_size : 5 prediction_size : 73680.0
6.786102062975027e-05
1.0


26542
label_size : 4 prediction_size : 73680.0
5.428881650380022e-05
1.0


71651
label_size : 2 prediction_size : 73680.0
2.714440825190011e-05
1.0


45696
label_size : 3 prediction_size : 73680.0
4.0716612377850165e-05
1.0


4253
label_size : 8 prediction_size : 73680.0
0.00010857763300760044
1.0


63461
label_size : 2 prediction_size : 73680.0
2.714440825190011e-05
1.0


572
label_size : 14 prediction_size : 73680.0
0.00019001085776330077
1.0


66460
label_size : 2 pr

In [46]:
print(len(precision_100), len(recall_100) )

100 100


In [47]:
np.mean(precision_100)

5.7410622410523634e-05

In [48]:
np.mean(recall_100)

1.0